In [1]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical
from keras import backend as K
#from theano import tensor as K

import numpy as np
import os
import PIL
from PIL import Image

Using Theano backend.


In [2]:
os.getcwd()

'/Users/Apple/Downloads/hand_gesture'

In [4]:
# VGG model definition
def VGG_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(224,224,3)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)

    return model

# Load the VGG weigths

#theano
model = VGG_16('vgg16_weights_th_dim_ordering_th_kernels.h5')

#tensorflow
model = VGG_16('vgg16_weights_tf_dim_ordering_tf_kernels.h5')

ValueError: The shape of the input to "Flatten" is not fully defined (got (512, 7, 0). Make sure to pass a complete "input_shape" or "batch_input_shape" argument to the first layer in your model.

In [6]:
size = 224

path = '/Users/Apple/Downloads/hand_gesture/Triesch'

def read_image(f):
    try: #read only jpgs (Add this)
        im = Image.open(f)
        im = im.resize((size, size), PIL.Image.NEAREST)
        im = np.asarray(im, dtype='float64')
        if len(im.shape)==2:
            im = im.reshape(im.shape[0],im.shape[1],1)
            im = im.repeat(3,axis=2)
        if im.shape[2]!=3:
            im = im[:,:,:3]
        im[:,:,0] -= 103.939
        im[:,:,1] -= 116.779
        im[:,:,2] -= 123.68
        im = im.reshape(1,224,224,3)
        return(im)
    except:
        print(f,im.shape)

In [8]:
# Basic Check of read_image
im = read_image('/Users/Apple/Downloads/hand_gesture/Triesch/bfritza2.pgm')
print(im.shape)
# model.layers[32]

(1, 224, 224, 3)


In [ ]:
# with a Sequential model, get output of only 33rd layer
get_33rd_layer_output = K.function([model.layers[0].input],
                                  [model.layers[32].output])
layer_output = get_33rd_layer_output([im])[0]
#print(layer_output.shape)

In [9]:
img_files = []
path = '/Users/Apple/Downloads/hand_gesture/Triesch/'
for root, dirs, files in os.walk(path):
   img_files.extend(files)
print(len(img_files))

719


In [10]:
n_files = len(img_files)
print('Total number of images:', n_files)

#train_size = 2000
#for demo
train_size = 10
sample_size = max(n_files,train_size)
print('Sample images taken:', sample_size)
sample = [img_files[i] for i in np.random.choice(len(img_files),size=sample_size, replace=False)]
print(sample[:10])
print(len(sample))

Total number of images: 719
Sample images taken: 719
['szadela2.pgm', 'haloosc1.pgm', 'haloosg2.pgm', 'hnevenl1.pgm', 'pleuchl3.pgm', 'hnevenb2.pgm', 'saginsa3.pgm', 'nkruegb2.pgm', 'hnevena1.pgm', 'ermaelb2.pgm']
719


In [22]:
file = "szadela2.pgm"
print(file[6])
y = []
for file in sample[:100]:
    y.extend()
print(y)
# l = [1, 2, 3, 4, 5]
# ['yes' if v == 1 else 'no' if v == 2 else 'idle' for v in l]
# ['yes', 'no', 'idle', 'idle', 'idle']

a
[0, 2, 4, 7, 7, 1, 0, 1, 0, 1, 7, 7, 4, 3, 1, 9, 9, 8, 1, 2, 7, 8, 4, 6, 8, 3, 4, 7, 8, 7, 8, 1, 2, 0, 3, 4, 7, 3, 7, 7, 0, 3, 4, 0, 8, 6, 4, 8, 1, 8, 4, 2, 2, 8, 0, 7, 2, 3, 4, 7, 1, 2, 4, 9, 3, 2, 6, 7, 3, 1, 8, 5, 4, 9, 4, 0, 0, 9, 5, 0, 7, 3, 7, 8, 9, 1, 9, 8, 5, 1, 2, 1, 8, 0, 3, 5, 1, 7, 6, 8]


In [24]:
# Read the train images 
i=0
x_train = []
y_train = []
print('Reading train images ...')
for file in sample[:14600]:
    if(i%600==0):
        print('Read {} images'.format(i))
    im = read_image(path+file)
    temp = get_33rd_layer_output([im])[0]
    x_train.append(temp[0])
    y_train.extend([0 if file[6]=='a' else 
              1 if file[6]=='b' else 
              2 if file[6]=='c' else 
              3 if file[6]=='d' else
              4 if file[6]=='g' else
              5 if file[6]=='h' else
              6 if file[6]=='i' else
              7 if file[6]=='l' else
              8 if file[6]=='v' else 9])
    i+=1

Reading train images ...
Read 0 images


NameError: name 'get_33rd_layer_output' is not defined

In [ ]:
# Read the val images 
j=0
x_val = []
y_val = []
print('Reading val images ...')
for file in sample[14600:]:
    if(j%600==0):
        print('Read {} images'.format(j))
    im = read_image('/home/s_jaysetty/Insofe_Cluster/train_samples_kaggle/'+file)
    temp = get_33rd_layer_output([im])[0]
    x_val.append(temp[0])
    y_val.extend([0 if file[:3]=='cat' else 1])
    j+=1

In [ ]:
print(type(x_train), type(y_train))
print(type(x_val), type(y_val))
# print(x_train.shape, y_train.shape)

x_train = np.array(x_train)
print(x_train.shape, 'x_train Shape')

# y_train = to_categorical(y_train)
y_train = np.array(y_train)
print(y_train.shape, 'y_train Shape')

x_val = np.array(x_val)
print(x_val.shape, 'x_train Shape')

# y_train = to_categorical(y_train)
y_val = np.array(y_val)
print(y_val.shape, 'y_val Shape')

print(type(x_train), type(y_train))
print(type(x_val), type(y_val))

In [ ]:
# Add the MLP Layer

mlp = Sequential()
mlp.add(Dense(256, input_dim=4096, activation='relu'))
mlp.add(Dropout(0.5))
mlp.add(Dense(256, activation='relu'))
mlp.add(Dropout(0.5))
mlp.add(Dense(1, activation='sigmoid'))#try dense as 1 and sigmoid
mlp.summary()

In [ ]:
#adam = Adam(lr=0.1)
mlp.compile(optimizer='Adam', 
            loss='binary_crossentropy', 
            metrics=['acc'])

mlp.fit(x_train, y_train, nb_epoch=20, validation_data=(x_val, y_val))

In [ ]:
img_files_test = []
for root, dirs, files in os.walk('/home/s_jaysetty/Insofe_Cluster/test/'):
    img_files_test.extend(files)

n_files_test = len(img_files_test)
print('Total number of images:', n_files_test)

test_size = 100
sample_size_test = max(n_files_test,test_size)
print('Sample images taken:', sample_size_test)
sample_test = [img_files_test[i] for i in np.random.choice(len(files),size=sample_size_test, replace=False)]
print(sample_test[:10])
print(len(sample_test))

i=0
x_test = []
y_test = []

print('Reading test images ...')
for file in img_files_test:
    if(i%200==0):
        print('Read {} images'.format(i))
    im = read_image('/home/s_jaysetty/Insofe_Cluster/test/'+file)
#     y_test.extend([0 if file[:3]=='cat' else 1])
    temp = get_33rd_layer_output([im])[0]
    x_test.append(temp[0])
    i+=1

x_test = np.array(x_test)
print(x_test.shape, 'x_test Shape')

In [ ]:
prediction = mlp.predict(x_test, batch_size=50)
print(prediction[:3])
print(prediction.shape)

In [ ]:
pred_class = mlp.predict_classes(x_test, batch_size=50)